## 1. Overview
### IVRMSE Baselines (BS/Black–76, Merton JD, Heston SV)

**Purpose.** Build transparent, reproducible baselines for option pricing models and report **IVRMSE** (implied-vol RMSE) in publication-style tables.

**Pipeline (per symbol):**  
1) **Data prep (per day):** clean quotes, pair Calls & Puts, infer **forward $F$** and **discount $DF$** via put–call parity; compute **market Black–76 IV** per call.  
2) **Calibration (per day × maturity bucket):**  
   • **BS/Black–76:** fit a *single constant* $\,\sigma\,$ in price space (cross-sectional SSE).  
   • **Merton JD:** fit $(\sigma,\;\lambda,\;\mu_J,\;\delta_J)$ by price SSE.  
   • **Heston SV:** fit $(\kappa,\;\theta,\;\sigma_v,\;\rho,\;v_0)$ by price SSE (characteristic function + Simpson integration).  
3) **Evaluation (per day × bucket):** price with the model, **invert to Black–76 IV**, and compute **IVRMSE** vs market IV, for slices **Whole / moneyness $<1$ / $>1$ / $>1.03$**.  
4) **Aggregation (over period):**  
   • **Primary** = equal-day mean IVRMSE (each day counts once)  
   • **Secondary** = pooled (contract-weighted) IVRMSE  
5) **Outputs:** daily table, equal-day mean, pooled, and a publication-style table (CSV + Markdown with bold minima).

**Data schema expected:** `date, act_symbol|symbol, expiration, strike, call_put, bid, ask`.


## 2. Black–76 helpers (pricing & IV inversion)

### Pricing (forward measure)
With forward $F$, strike $K$, maturity $\tau$, rate $r$, and volatility $\sigma$:
$$
C_{\text{B76}} \;=\; DF\,\big(F\,\Phi(d_1) - K\,\Phi(d_2)\big), \qquad
d_1=\frac{\ln(F/K)+\tfrac12\sigma^2\tau}{\sigma\sqrt{\tau}},\quad
d_2=d_1-\sigma\sqrt{\tau},
$$
where $DF=e^{-r\tau}$ and $\Phi(\cdot)$ is the standard normal CDF.  
When $F$ and $DF$ are inferred via put–call parity, Black–76 coincides numerically with Black–Scholes on the same inputs.  
**Edge cases:** if $\tau\le 0$ or $\sigma\le 0$, use the discounted intrinsic value $DF\cdot\max(F-K,0)$.

### IV inversion (market price → implied $\sigma$)
Given the **market call price** $C^{\text{mkt}}$ (we use the mid $(\text{bid}+\text{ask})/2$), clamp to the intrinsic value:
$$
\tilde C \;=\; \max\!\big(C^{\text{mkt}},\; DF\cdot\max(F-K,0)\big).
$$
Solve for $\sigma$ in
$$
C_{\text{B76}}(F,K,\tau,r,\sigma) \;=\; \tilde C
$$
using **bisection with bracket expansion**:
start with $[\sigma_{\min},\sigma_{\max}] = [10^{-4},\,5]$, expand $\sigma_{\max}$ until a sign change is found (or a safety cap), and stop when $|C_{\text{B76}}-\tilde C|$ is below tolerance or the interval is sufficiently small.  
The solution is the market implied volatility $\sigma_{\text{mkt}}$ used downstream for IVRMSE.


In [1]:
from __future__ import annotations

import math
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple, Optional
from pathlib import Path

# Try SciPy for calibration; fall back to coarse grid if missing
try:
    from scipy.optimize import minimize
    _HAVE_SCIPY = True
except Exception:
    _HAVE_SCIPY = False

# ----------------------------
# Black–76 pricing & IV inversion
# ----------------------------

def _norm_cdf(x: float) -> float:
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

def b76_price_call(F: float, K: float, tau: float, r: float, sigma: float) -> float:
    """Black–76 call price (also BS if F, DF inferred via parity)."""
    if tau <= 0 or K <= 0 or F <= 0:
        DF = math.exp(-r * max(tau, 0.0))
        return DF * max(F - K, 0.0)
    if sigma <= 0:
        DF = math.exp(-r * tau)
        return DF * max(F - K, 0.0)
    v = sigma * math.sqrt(tau)
    d1 = (math.log(F / K) + 0.5 * sigma * sigma * tau) / v
    d2 = d1 - v
    DF = math.exp(-r * tau)
    return DF * (F * _norm_cdf(d1) - K * _norm_cdf(d2))

def b76_iv_from_price(target: float, F: float, K: float, tau: float, r: float,
                      tol: float = 1e-8, max_iter: int = 100,
                      lo: float = 1e-4, hi: float = 5.0) -> Optional[float]:
    """Robust bisection with bracket expansion; clamps target to intrinsic."""
    if not (F > 0 and K > 0 and tau >= 0 and math.isfinite(target)):
        return None
    DF = math.exp(-r * tau)
    intrinsic = DF * max(F - K, 0.0)
    t = max(target, intrinsic)

    def f(sig: float) -> float:
        return b76_price_call(F, K, tau, r, sig) - t

    a, b = lo, hi
    fa, fb = f(a), f(b)
    k = 0
    while fa * fb > 0 and k < 25:
        b *= 1.5
        fb = f(b)
        k += 1
        if b > 50:
            break
    if fa * fb > 0:
        return None

    for _ in range(max_iter):
        c = 0.5 * (a + b)
        fc = f(c)
        if abs(fc) < tol or 0.5 * (b - a) < 1e-8:
            return c
        if fa * fc <= 0:
            b, fb = c, fc
        else:
            a, fa = c, fc
    return 0.5 * (a + b)

## 3. Put–call parity → infer forward $F$ and discount $DF$

For all strikes on the same **(day, symbol, expiry)**, with midprices $C$ (call) and $P$ (put):
$$
C - P \;=\; DF\,(F - K) \;=\; a \;+\; b(-K).
$$

**OLS regression across strikes.**  
Let $y_i = C_i - P_i$ and $x_i = -K_i$. Fit the line
$$
y_i \;=\; a \;+\; b\,x_i
$$
by least squares using all **paired** call–put quotes for that expiry.

**Recover forward and discount:**
- **Discount factor:** $DF = b$  
- **Forward:** $F = \dfrac{a}{DF}$  
- **Short rate:** $r = -\dfrac{\ln(DF)}{\tau}$ (with $\tau$ in years)

**Practical notes.**
- Require at least **two** call–put pairs for the regression.  
- Drop the expiry if the fit gives nonpositive $DF$ or nonpositive $F$.  
- With only **one** pair available, a safe fallback is $DF=1$ and $F=C-P+K$ (used only when necessary).

This parity step produces $(F, DF, r)$ per expiry, which we then use for pricing and IV inversion downstream.

In [2]:
# ----------------------------
# Put–call parity: infer F and DF
# ----------------------------

def parity_infer_F_DF(df_pairs: pd.DataFrame) -> Optional[Tuple[float, float]]:
    """
    Infer forward F and discount DF from (C - P) vs K.
    y = C - P = DF*(F - K) = a + b*(-K)  ->  b = DF,  F = a/DF.
    """
    y = (df_pairs["C_mid"].values - df_pairs["P_mid"].values).astype(float)
    X = df_pairs["strike"].values.astype(float)
    if len(y) < 2:
        # fallback with single pair
        K0 = float(df_pairs["strike"].iloc[0])
        DF = 1.0
        F = float(y[0] + K0)
        return F, DF
    Xmat = np.column_stack([np.ones_like(X), -X])
    a, b = np.linalg.lstsq(Xmat, y, rcond=None)[0]
    DF = float(b)
    if not np.isfinite(DF) or DF <= 0:
        return None
    F = float(a / DF)
    if not np.isfinite(F) or F <= 0:
        return None
    return F, DF

## 4. Maturity bucketing & one-parameter BS fit

**Bucketing by maturity.**  
Map time-to-expiry $\tau$ (in years) to the nearest center in **calendar days** (e.g., 14d / 28d / 56d).  
This groups strikes for a given **(day, symbol)** into cross-sections with similar maturities.

**Single-parameter BS baseline (per bucket).**  
Fit a **constant volatility** $\hat{\sigma}$ by minimizing **call price SSE** over all strikes in the bucket:
$$
\hat{\sigma}
\;=\;
\arg\min_{\sigma>0}\;
\sum_{i\in \text{bucket}}
\Big(
C_{\text{B76}}(F_i,K_i,\tau_i,r_i,\sigma)\;-\;C^{\text{mkt}}_i
\Big)^2.
$$

**Numerics.**  
- Start with a **log-spaced grid** for $\sigma$ (e.g., $[0.05,\,2.0]$); pick the best grid point.  
- **Refine** via **golden-section search** on a bracket around the best grid value.  
- Edge cases: if a candidate $\sigma\le 0$, fall back to discounted intrinsic $DF\cdot\max(F-K,0)$ inside pricing.

**Evaluation → IVRMSE slices.**  
Treat the fitted $\hat{\sigma}$ as the model’s **implied vol** and compare to **market B76 IVs** within the bucket:
- **Whole**: all strikes,  
- **$m<1$**: $m=K/F<1$ (OTM calls / ITM puts),  
- **$m>1$**: $K/F>1$,  
- **$m>1.03$**: deeper OTM calls.

For a slice $\mathcal{S}$, the reported metric is
$$
\text{IVRMSE}_\mathcal{S}
\;=\;
\sqrt{\;\frac{1}{|\mathcal{S}|}\sum_{i\in\mathcal{S}}
\big(\hat{\sigma}\;-\;\sigma^{\text{mkt}}_i\big)^2\;}\;\times 1000,
$$
i.e., RMSE in **implied-vol space**, scaled by **1000** as in the reference tables.


In [3]:
# ----------------------------
# Maturity bucketing & σ fit
# ----------------------------

def assign_bucket(tau_years: float, centers_days: List[int] = [14, 28, 56]) -> str:
    """Map τ (years) to nearest maturity center in days: '14d'/'28d'/'56d'."""
    days = tau_years * 365.0
    idx = int(np.argmin([abs(days - c) for c in centers_days]))
    return f"{centers_days[idx]}d"

def fit_sigma_bucket(df_bucket: pd.DataFrame) -> float:
    """
    1-parameter BS/B76 baseline: choose σ minimizing price SSE.
    Coarse grid (geomspace) + golden-section refinement.
    """
    if len(df_bucket) == 0:
        return np.nan

    def sse(sig: float) -> float:
        if sig <= 0:
            return 1e18
        prices = df_bucket.apply(
            lambda r: b76_price_call(r["F"], r["strike"], r["tau"], r["r"], sig), axis=1
        ).values
        err = prices - df_bucket["C_mid"].values
        return float(np.dot(err, err))

    grid = np.geomspace(0.05, 2.0, 40)
    best = min(grid, key=sse)
    a = max(best / 3, 1e-4)
    b = min(best * 3, 5.0)
    phi = (1 + np.sqrt(5)) / 2
    invphi = 1 / phi
    c = b - (b - a) * invphi
    d = a + (b - a) * invphi
    fc, fd = sse(c), sse(d)
    for _ in range(60):
        if fc < fd:
            b, d, fd = d, c, fc
            c = b - (b - a) * invphi
            fc = sse(c)
        else:
            a, c, fc = c, d, fd
            d = a + (b - a) * invphi
            fd = sse(d)
    return float((a + b) / 2)

## 5. Merton Jump–Diffusion (JD)

**Model.** Log-returns combine a continuous diffusion (volatility $\sigma$) with **compound Poisson jumps** of intensity $\lambda$ and jump sizes $Y\sim\mathcal{N}(\mu_J,\delta_J^2)$.  
Under the risk-neutral forward measure (we work with forward $F$ and discount $DF=e^{-r\tau}$), the call price is a **Poisson mixture** of Black–76 prices:

$$
C_{\text{JD}} \;=\; 
\sum_{n=0}^{\infty} p_n \;
C_{\text{B76}}\!\big(F_n,\;K,\;\tau,\;r,\;\sigma_n\big),
\qquad 
p_n \;=\; e^{-\lambda\tau}\frac{(\lambda\tau)^n}{n!}.
$$

**Compensation (drift fix) and mixture terms.**  
Because jumps are modeled in log-price, we compensate the forward so the overall drift stays risk-neutral:
$$
k \;=\; \mathbb{E}[e^{Y}] - 1 \;=\; e^{\mu_J+\tfrac12 \delta_J^2} - 1, 
\qquad
F_{\text{adj}} \;=\; F\; e^{-\lambda k \tau}.
$$
For the $n$-jump component,
$$
F_n \;=\; F_{\text{adj}} \, e^{n\mu_J}, 
\qquad 
\sigma_n^2 \;=\; \sigma^2 \;+\; \frac{n\,\delta_J^2}{\tau}.
$$

**Calibration (per day × bucket).**  
Estimate $\theta=(\sigma,\lambda,\mu_J,\delta_J)$ by minimizing **call price SSE** on the bucket cross-section:
$$
\min_{\theta}\; 
\sum_{i\in \text{bucket}}
\Big(
C_{\text{JD}}(F_i,K_i,\tau_i,r_i;\theta) - C^{\text{mkt}}_i
\Big)^2.
$$

**Numerics.**
- **Poisson truncation:** sum terms until the cumulative mass exceeds $1-\varepsilon$ (e.g., $\varepsilon=10^{-8}$), or up to a cap $n_{\max}$ (e.g., 50–80).  
- **Initialization (robust):** $\sigma \approx \text{median ATM IV}$, $\lambda\!\approx\!0.1$, $\mu_J\!\approx\!-0.02$, $\delta_J\!\approx\!0.10$.  
- **Typical bounds:** $\sigma\in[0.01,3]$, $\lambda\in[0,5]$, $\mu_J\in[-0.5,0.5]$, $\delta_J\in[0.01,1]$.  
- **Identifiability note:** for short maturities, $(\sigma,\delta_J,\lambda)$ can trade off; use sensible bounds and regularization (if needed).

**Evaluation for IVRMSE (downstream).**  
After calibration, price each strike with $C_{\text{JD}}$, **invert to B76 IV**, and compare to market IV to form IVRMSE slices (Whole, $m<1$, $m>1$, $m>1.03$), consistent with the BS baseline.

In [4]:
# ============================================================
# Merton Jump–Diffusion (JD)
# ============================================================

def merton_price_call_b76(F: float, K: float, tau: float, r: float,
                          sigma: float, lam: float, muJ: float, deltaJ: float,
                          eps_tail: float = 1e-8, n_max: int = 80) -> float:
    """
    Merton JD via Poisson mixture of B76 prices.
    Compensation k = E[e^Y]-1 = exp(muJ + 0.5*deltaJ^2) - 1
    Use F_adj = F * exp(-lam * k * tau).
    For n jumps: F_n = F_adj * exp(n*muJ), sigma_n^2 = sigma^2 + n*deltaJ^2 / tau.
    """
    if tau <= 0:
        DF = math.exp(-r * max(tau, 0.0))
        return DF * max(F - K, 0.0)

    k = math.exp(muJ + 0.5 * deltaJ * deltaJ) - 1.0
    F_adj = F * math.exp(-lam * k * tau)
    L = lam * tau

    p = math.exp(-L)  # p0
    price = p * b76_price_call(F_adj, K, tau, r, sigma)
    cum = p
    n = 0
    while cum < 1 - eps_tail and n < n_max:
        n += 1
        p = p * (L / n)  # p_n
        sigma_n = math.sqrt(sigma * sigma + (n * deltaJ * deltaJ) / max(tau, 1e-12))
        F_n = F_adj * math.exp(n * muJ)
        price += p * b76_price_call(F_n, K, tau, r, sigma_n)
        cum += p
    return float(price)

## 6. Heston Stochastic Volatility (SV) via characteristic function

**Model (under risk-neutral dynamics).**  
Stock variance follows
$$
dv_t \;=\; \kappa(\theta - v_t)\,dt \;+\; \sigma_v\sqrt{v_t}\,dW_t^{(2)},
\qquad
dW_t^{(1)}\,dW_t^{(2)} \;=\; \rho\,dt,
$$
and $dS_t = S_t\sqrt{v_t}\,dW_t^{(1)}$ after the drift is adjusted for risk neutrality.  
We work under the **forward measure** using parity-inferred inputs: set $S_0 \to F$ and $q=r$ so that pricing uses the forward $F$ and discount $DF=e^{-r\tau}$.

**Pricing via $P_1/P_2$ integrals.**  
The Heston call price is
$$
C \;=\; DF\,\big(F\,P_1 \;-\; K\,P_2\big),
$$
with
$$
P_2 \;=\; \tfrac12 \;+\; \tfrac{1}{\pi}\int_0^\infty 
\Re\!\left\{ e^{-iu\ln K}\,\frac{\phi(u)}{iu} \right\}\,du,
\qquad
P_1 \;=\; \tfrac12 \;+\; \tfrac{1}{\pi}\int_0^\infty 
\Re\!\left\{ e^{-iu\ln K}\,\frac{\phi(u-i)}{iu\,\phi(-i)} \right\}\,du.
$$

**Characteristic function (little Heston trap).**  
Let $x=\ln F$, $i=\sqrt{-1}$, and define
$$
d(u) \;=\; \sqrt{(\rho\,\sigma_v\,iu - \kappa)^2 + \sigma_v^2\,(iu + u^2)},\qquad
g(u) \;=\; \frac{\kappa - \rho\,\sigma_v\,iu - d(u)}{\kappa - \rho\,\sigma_v\,iu + d(u)}.
$$
Then
$$
\begin{aligned}
C(u) &= \frac{\kappa\theta}{\sigma_v^2}
\Big[(\kappa - \rho\,\sigma_v\,iu - d(u))\,\tau
\;-\; 2\ln\!\Big(\frac{1 - g(u)\,e^{-d(u)\tau}}{1 - g(u)}\Big)\Big],\\[4pt]
D(u) &= \frac{\kappa - \rho\,\sigma_v\,iu - d(u)}{\sigma_v^2}
\cdot \frac{1 - e^{-d(u)\tau}}{1 - g(u)\,e^{-d(u)\tau}},
\end{aligned}
$$
and the **characteristic function** of $\ln S_T$ (under the forward measure) is
$$
\phi(u) \;=\; \exp\!\big( C(u) + D(u)\,v_0 + iu\,x \big).
$$

**Numerics.**  
- Compute $P_1, P_2$ by **Simpson’s rule** on a uniform grid $u\in(0,u_{\max})$ (e.g., $u_{\max}\in[80,100]$, 400–600 points).  
- Use the “little trap” form above to avoid branch-cut issues for the complex logarithm.  
- Discount with $DF=e^{-r\tau}$ where $r$ is inferred from parity via $DF$.

**Calibration (per day × maturity bucket).**  
Fit $\Theta=(\kappa,\theta,\sigma_v,\rho,v_0)$ by minimizing **call price SSE** over the bucket cross-section:
$$
\min_{\Theta}\; \sum_{i\in\text{bucket}}
\Big(C_{\text{Heston}}(F_i,K_i,\tau_i,r_i;\Theta) - C^{\text{mkt}}_i\Big)^2.
$$
Practical choices:
- **Initialization:** $v_0 \approx (\text{ATM IV})^2$, $\kappa\approx 2$, $\theta\approx v_0$, $\sigma_v\approx 0.5$, $\rho\approx -0.5$.  
- **Bounds:** $\kappa>0$, $\theta>0$, $\sigma_v>0$, $\rho\in[-0.999,0]$, $v_0>0$.

**Evaluation for IVRMSE.**  
After calibration, price each strike with Heston, **invert the price to a Black–76 IV**, and compare to the market B76 IV to form IVRMSE slices (Whole, $m<1$, $m>1$, $m>1.03$), consistent with BS and JD baselines.


In [5]:
# ============================================================
# Merton Jump–Diffusion (JD)
# ============================================================

def merton_price_call_b76(F: float, K: float, tau: float, r: float,
                          sigma: float, lam: float, muJ: float, deltaJ: float,
                          eps_tail: float = 1e-8, n_max: int = 80) -> float:
    """
    Merton JD via Poisson mixture of B76 prices.
    Compensation k = E[e^Y]-1 = exp(muJ + 0.5*deltaJ^2) - 1
    Use F_adj = F * exp(-lam * k * tau).
    For n jumps: F_n = F_adj * exp(n*muJ), sigma_n^2 = sigma^2 + n*deltaJ^2 / tau.
    """
    if tau <= 0:
        DF = math.exp(-r * max(tau, 0.0))
        return DF * max(F - K, 0.0)

    k = math.exp(muJ + 0.5 * deltaJ * deltaJ) - 1.0
    F_adj = F * math.exp(-lam * k * tau)
    L = lam * tau

    p = math.exp(-L)  # p0
    price = p * b76_price_call(F_adj, K, tau, r, sigma)
    cum = p
    n = 0
    while cum < 1 - eps_tail and n < n_max:
        n += 1
        p = p * (L / n)  # p_n
        sigma_n = math.sqrt(sigma * sigma + (n * deltaJ * deltaJ) / max(tau, 1e-12))
        F_n = F_adj * math.exp(n * muJ)
        price += p * b76_price_call(F_n, K, tau, r, sigma_n)
        cum += p
    return float(price)

# ============================================================
# Heston Stochastic Volatility (SV)
# ============================================================

def _heston_cf(u: np.ndarray, F: float, tau: float,
               kappa: float, theta: float, sigma_v: float, rho: float, v0: float) -> np.ndarray:
    """
    Heston characteristic function φ(u) for log-price under forward measure
    (set S0 = F and q = r). Implements the "little Heston trap" form.
    Vectorized over u (real or complex).
    """
    x = math.log(F)
    iu = 1j * u
    d = np.sqrt((rho * sigma_v * iu - kappa)**2 + sigma_v**2 * (iu + u*u))
    g = (kappa - rho * sigma_v * iu - d) / (kappa - rho * sigma_v * iu + d)
    exp_dt = np.exp(-d * tau)
    C = (kappa * theta / (sigma_v**2)) * ((kappa - rho * sigma_v * iu - d) * tau - 2.0 * np.log((1 - g * exp_dt) / (1 - g)))
    D = ((kappa - rho * sigma_v * iu - d) / (sigma_v**2)) * ((1 - exp_dt) / (1 - g * exp_dt))
    return np.exp(C + D * v0 + iu * x)

def _simpson_integral(fx: np.ndarray, dx: float) -> float:
    """Simpson’s rule; falls back to trapz if <3 points. Ensures odd #points."""
    n = len(fx)
    if n < 3:
        return float(np.trapz(fx, dx=dx))
    if n % 2 == 0:
        fx = fx[:-1]; n -= 1
    S = fx[0] + fx[-1] + 4.0 * fx[1:-1:2].sum() + 2.0 * fx[2:-2:2].sum()
    return float((dx / 3.0) * S)

def _heston_prob(F: float, K: float, tau: float, params: Dict[str, float],
                 j: int, u_max: float = 100.0, n_points: int = 501) -> float:
    """
    Risk-neutral probabilities P1 (j=1) and P2 (j=2):
      P2 = 1/2 + 1/π ∫_0^∞ Re[ e^{-i u ln K} φ(u) / (i u) ] du
      P1 = 1/2 + 1/π ∫_0^∞ Re[ e^{-i u ln K} φ(u - i) / (i u * φ(-i)) ] du
    """
    kappa, theta, sigma_v, rho, v0 = params["kappa"], params["theta"], params["sigma_v"], params["rho"], params["v0"]
    lnK = math.log(K)
    u = np.linspace(1e-6, u_max, n_points)
    du = u[1] - u[0]

    if j == 2:
        phi = _heston_cf(u, F, tau, kappa, theta, sigma_v, rho, v0)
        integrand = np.real(np.exp(-1j * u * lnK) * phi / (1j * u))
    else:
        phi_shift = _heston_cf(u - 1j, F, tau, kappa, theta, sigma_v, rho, v0)
        phi_mi = _heston_cf(np.array([-1j]), F, tau, kappa, theta, sigma_v, rho, v0)[0]
        integrand = np.real(np.exp(-1j * u * lnK) * (phi_shift / (1j * u * phi_mi)))

    integral = _simpson_integral(integrand, du)
    return float(0.5 + (1.0 / math.pi) * integral)

def heston_price_call(F: float, K: float, tau: float, r: float, params: Dict[str, float],
                      u_max: float = 100.0, n_points: int = 501) -> float:
    """Heston price via P1/P2 under forward measure."""
    DF = math.exp(-r * tau)
    P1 = _heston_prob(F, K, tau, params, j=1, u_max=u_max, n_points=n_points)
    P2 = _heston_prob(F, K, tau, params, j=2, u_max=u_max, n_points=n_points)
    return DF * (F * P1 - K * P2)

## 7. Data preparation → Calls with market Black–76 IVs

**Goal.** From raw quotes on a given **(day, symbol)**, build a clean cross-section of calls with:
forward $F$, discount $DF$, short rate $r$, year-fraction $\tau$, bucket label, and **market B76 IV** per strike.

### Inputs (expected columns)
`date, act_symbol|symbol, expiration, strike, call_put, bid, ask`.

### Step 1 — Basic cleaning
- Parse `date` and `expiration`; compute **calendar-day** year fraction  
  $$\tau \;=\; \frac{\text{expiration} - \text{date}}{365}.$$
- Require valid quotes: `bid ≥ 0`, `ask ≥ bid`.  
- Define midprice  
  $$\text{mid} \;=\; \frac{\text{bid} + \text{ask}}{2} \;>\; 0.$$
- Optional guardrails:
  - drop rows with $\tau \le 0$,
  - apply a **$\tau$ floor** (e.g., $\ge$ 3 days) to exclude micro-expiries.

### Step 2 — Pair calls and puts by strike
- Pivot by (`date`, `symbol`, `expiration`, `tau`, `strike`) to obtain both **Call** and **Put** mids on the same grid.  
- Keep only **pairs** (both sides present) — these are used for parity.

### Step 3 — Put–call parity per expiry → $(F, DF, r)$
- For each expiry on that day, regress across strikes:
  $$
  C - P \;=\; DF\,(F - K) \;=\; a + b(-K).
  $$
  Then **slope** $b=DF$, **intercept** $a=DF\cdot F \Rightarrow F=a/DF$.
- Recover the short rate from the discount factor:
  $$ r \;=\; -\frac{\ln(DF)}{\tau}. $$
- Require at least **two** call–put pairs; discard if $DF \le 0$ or $F \le 0$.  
  *(Fallback when only one pair exists: set $DF=1$, $F=C-P+K$ — used sparingly.)*

### Step 4 — Market B76 IVs (calls only)
- For each **call** row at that expiry, invert the Black–76 price to get
  $$\sigma_i^{\text{mkt}} \quad \text{such that}\quad
    C_{\text{B76}}(F_i,K_i,\tau_i,r_i,\sigma_i^{\text{mkt}}) \;=\; \max\!\big(\text{mid}_i,\; DF_i\max(F_i-K_i,0)\big).$$
  (Clamp to intrinsic ensures feasibility; solve via bisection with bracket expansion.)

### Step 5 — Moneyness & maturity buckets
- Define moneyness  
  $$ m \;=\; \frac{K}{F}. $$
- Map $\tau$ to the nearest **bucket center** in calendar days (e.g., **14d**, **28d**, **56d**).  
  These buckets are used for **same-maturity** cross-sectional calibration.

### Output of this stage
A per-day DataFrame of calls with columns like:  
`date, symbol, expiration, tau, strike, C_mid, F, DF, r, sigma_mkt_b76, moneyness_F, bucket`  
This is the input for calibration and IVRMSE evaluation in later sections.

In [6]:
# ============================================================
# Data prep: one day & symbol -> calls with market IVs
# ============================================================

def prepare_calls_one_day_symbol(df_day_symbol: pd.DataFrame,
                                 min_parity_pairs: int = 2,
                                 tau_floor_days: int = 0) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Input (one trading day × one symbol quotes) →
      calls_df with: F, DF, r, tau, bucket, market B76 IV (calls only)
      parity_df with per-expiry inferred (F, DF, r).
    """
    df = df_day_symbol.copy()
    # normalize schema (accept 'act_symbol' or 'symbol')
    rename_map = {"date":"date","expiration":"expiration","strike":"strike","call_put":"cp","bid":"bid","ask":"ask"}
    if "act_symbol" in df.columns: rename_map["act_symbol"] = "symbol"
    elif "symbol" in df.columns:   rename_map["symbol"] = "symbol"
    else: raise ValueError("Expected 'act_symbol' or 'symbol' column.")
    df = df.rename(columns=rename_map)

    df["date"] = pd.to_datetime(df["date"])
    df["expiration"] = pd.to_datetime(df["expiration"])
    df["tau"] = (df["expiration"] - df["date"]).dt.days / 365.0

    df["bid"] = pd.to_numeric(df["bid"], errors="coerce")
    df["ask"] = pd.to_numeric(df["ask"], errors="coerce")
    df["mid"] = (df["bid"].clip(lower=0) + df["ask"].clip(lower=0)) / 2.0

    # cleaning & optional τ floor
    df = df[df["mid"].notna() & (df["mid"] > 0) & (df["ask"] >= df["bid"]) & (df["bid"] >= 0) & (df["tau"] > 0)]
    if tau_floor_days and tau_floor_days > 0:
        df = df[df["tau"] * 365.0 >= tau_floor_days]
    if df.empty: return pd.DataFrame(), pd.DataFrame()

    # pair calls & puts
    pvt = df.pivot_table(index=["date","symbol","expiration","tau","strike"],
                         columns="cp", values="mid", aggfunc="first").reset_index()
    pvt = pvt.rename(columns={"Call":"C_mid","Put":"P_mid"})
    pairs = pvt.dropna(subset=["C_mid","P_mid"])
    if pairs.empty:
        return pd.DataFrame(), pd.DataFrame()

    # infer F, DF, r per expiry
    recs, parity_rows = [], []
    for (date, sym, exp), g in pairs.groupby(["date","symbol","expiration"]):
        if len(g) < min_parity_pairs:
            continue
        tau = float(g["tau"].iloc[0])
        res = parity_infer_F_DF(g[["strike","C_mid","P_mid"]])
        if res is None:
            continue
        F, DF = res
        r = -math.log(max(DF, 1e-12)) / max(tau, 1e-12)

        gg = pvt[(pvt["date"]==date)&(pvt["symbol"]==sym)&(pvt["expiration"]==exp)].copy()
        gg["F"], gg["DF"], gg["r"] = F, DF, r
        recs.append(gg)
        parity_rows.append({"date":date,"symbol":sym,"expiration":exp,"tau":tau,"F":F,"DF":DF,"r":r,"n_pairs":len(g)})

    if not recs:
        return pd.DataFrame(), pd.DataFrame()

    calls = pd.concat(recs, ignore_index=True)
    parity_df = pd.DataFrame(parity_rows)

    # market B76 IVs (calls only)
    calls = calls[calls["C_mid"].notna()].copy()
    calls["sigma_mkt_b76"] = calls.apply(
        lambda r: b76_iv_from_price(r["C_mid"], r["F"], r["strike"], r["tau"], r["r"]), axis=1
    )
    calls = calls.dropna(subset=["sigma_mkt_b76"]).copy()
    if calls.empty: return pd.DataFrame(), parity_df

    calls["moneyness_F"] = calls["strike"] / calls["F"]
    # bucket default; caller may overwrite with custom centers
    calls["bucket"] = calls["tau"].apply(assign_bucket)
    return calls, parity_df

## 8. Calibration helpers (SciPy + fallback)

We fit model parameters **per (day × maturity bucket)** by minimizing **call price SSE** over the cross-section.

### Objective (generic)
Given fixed inputs $\{F_i, K_i, \tau_i, r_i, C^{\text{mkt}}_i\}_{i\in\mathcal{B}}$ for a bucket $\mathcal{B}$ and a model $M(\cdot;\,\theta)$,
$$
\min_{\theta}\;\;\text{SSE}(\theta)
\;=\;
\sum_{i\in\mathcal{B}}
\Big(M(F_i,K_i,\tau_i,r_i;\theta) - C^{\text{mkt}}_i\Big)^2.
$$
Each model (JD/Heston) provides its **parameter vector** $\theta$, **bounds**, and an **initial guess** based on ATM IV.

### Solver strategy
- **Primary (if SciPy is available):** use **L-BFGS-B** with box constraints (bounds), `maxiter≈200`, and early stop on small gradient/step.
- **Fallback (no SciPy):** evaluate a **small local grid** around the initial guess (±20% of each bounded range, ~7 points per dim) and **pick the best grid point** (lowest SSE).

### Numerical guards (penalty = large SSE)
- Enforce parameter domains (e.g., $\sigma>0$, $\lambda\ge0$, $\delta_J>0$, $\kappa>0$, $\theta>0$, $\sigma_v>0$, $\rho\in[-0.999,0]$, $v_0>0$).
- If pricing fails/overflows, return a large SSE so the optimizer avoids that region.
- **JD:** truncate Poisson mixture when cumulative mass $>1-\varepsilon$ (e.g., $\varepsilon=10^{-8}$) or at $n_{\max}$ (50–80).
- **Heston:** use the **little Heston trap** CF and **Simpson’s rule** on a stable grid $u\in(0,u_{\max})$.

### Outputs (per bucket)
- **Params:** best-fit $\hat\theta$ as a dict.  
- **Fit quality:** final SSE value (for diagnostics).  
These parameters are then used to **price the cross-section**, invert to **B76 IV**, and compute the **IVRMSE** slices.

In [7]:
# ============================================================
# Calibration helpers (JD & Heston)
# ============================================================

def _minimize(func, x0, bounds):
    """SciPy L-BFGS-B; fall back to a small local grid around x0 if SciPy missing."""
    if _HAVE_SCIPY:
        res = minimize(func, x0, method="L-BFGS-B", bounds=bounds, options={"maxiter": 200})
        return (res.x if res.success else x0)
    x0 = np.array(x0, dtype=float)
    grids = []
    for (lo, hi), xi in zip(bounds, x0):
        span = (hi - lo)
        g = np.linspace(max(lo, xi - 0.2*span), min(hi, xi + 0.2*span), 7)
        grids.append(g)
    mesh = np.meshgrid(*grids, indexing="ij")
    cand = np.stack([m.ravel() for m in mesh], axis=1)
    vals = np.array([func(p) for p in cand])
    return cand[int(np.argmin(vals))]

def calibrate_jd_bucket(calls_bucket: pd.DataFrame) -> Tuple[Dict[str, float], float]:
    """Fit JD params (sigma, lam, muJ, deltaJ) by price SSE on the bucket cross-section."""
    if calls_bucket.empty: return {}, np.inf
    atm_iv = float(np.median(calls_bucket["sigma_mkt_b76"].values))

    def sse_vec(p):
        sigma, lam, muJ, dJ = p
        if sigma <= 0 or lam < 0 or dJ <= 0: return 1e18
        prices = np.array([
            merton_price_call_b76(float(r0["F"]), float(r0["strike"]), float(r0["tau"]), float(r0["r"]),
                                  sigma, lam, muJ, dJ)
            for _, r0 in calls_bucket.iterrows()
        ], dtype=float)
        err = prices - calls_bucket["C_mid"].values
        return float(np.dot(err, err))

    x0 = np.array([max(0.02, atm_iv), 0.1, -0.02, 0.10], dtype=float)
    bounds = [(0.01, 3.0), (0.0, 5.0), (-0.5, 0.5), (0.01, 1.0)]
    p = _minimize(sse_vec, x0, bounds)
    params = {"sigma": float(p[0]), "lam": float(p[1]), "muJ": float(p[2]), "deltaJ": float(p[3])}
    sse_final = sse_vec([params["sigma"], params["lam"], params["muJ"], params["deltaJ"]])
    return params, sse_final

def calibrate_heston_bucket(calls_bucket: pd.DataFrame,
                            u_max: float = 100.0, n_points: int = 501) -> Tuple[Dict[str, float], float]:
    """Fit Heston params (kappa, theta, sigma_v, rho, v0) by price SSE on the bucket cross-section."""
    if calls_bucket.empty: return {}, np.inf
    atm_iv = float(np.median(calls_bucket["sigma_mkt_b76"].values))

    def sse_vec(p):
        kappa, theta, sigma_v, rho, v0 = p
        if kappa <= 0 or theta <= 0 or sigma_v <= 0 or not (-0.999 <= rho <= 0.0) or v0 <= 0:
            return 1e18
        params = {"kappa":kappa, "theta":theta, "sigma_v":sigma_v, "rho":rho, "v0":v0}
        prices = np.array([
            heston_price_call(float(r0["F"]), float(r0["strike"]), float(r0["tau"]), float(r0["r"]), params,
                              u_max=u_max, n_points=n_points)
            for _, r0 in calls_bucket.iterrows()
        ], dtype=float)
        err = prices - calls_bucket["C_mid"].values
        return float(np.dot(err, err))

    v0_0 = max(1e-4, atm_iv*atm_iv)
    x0 = np.array([2.0, max(1e-4, v0_0), 0.5, -0.5, v0_0], dtype=float)
    bounds = [(0.05, 10.0), (1e-4, 2.0), (1e-3, 3.0), (-0.999, 0.0), (1e-4, 3.0)]
    p = _minimize(sse_vec, x0, bounds)
    params = {"kappa": float(p[0]), "theta": float(p[1]), "sigma_v": float(p[2]), "rho": float(p[3]), "v0": float(p[4])}
    sse_final = sse_vec([params["kappa"], params["theta"], params["sigma_v"], params["rho"], params["v0"]])
    return params, sse_final

## 9. IVRMSE computation & aggregation

**Per (day × bucket):**  
- **BS/Black–76.** The model IV is the **single fitted** $\hat\sigma$ for that bucket. Compare directly to market IVs:
$$
\text{IVRMSE}_{\text{BS},\,b,d}
=\sqrt{\frac{1}{n_{b,d}}\sum_{i\in(b,d)}
\big(\hat\sigma-\sigma^{\text{mkt}}_i\big)^2}\times 1000.
$$

- **JD / Heston.** For each strike $i$, **price** with the calibrated model, then **invert** that price to a Black–76 IV $\sigma^{\text{mdl}}_i$, and compare to the market IV:
$$
\text{IVRMSE}_{\text{mdl},\,b,d}
=\sqrt{\frac{1}{n_{b,d}}\sum_{i\in(b,d)}
\big(\sigma^{\text{mdl}}_i-\sigma^{\text{mkt}}_i\big)^2}\times 1000,
\quad \text{mdl}\in\{\text{JD},\text{Heston}\}.
$$

Here $n_{b,d}$ is the number of **usable call contracts** in bucket $b$ on day $d$ (after cleaning and successful IV inversion).

**Report slices (computed separately):**  
- **Whole**: all strikes in the bucket.  
- **$m<1$**: $m=K/F<1$.  
- **$m>1$**: $m=K/F>1$.  
- **$m>1.03$**: deeper OTM calls.

---

### Aggregation over a period (primary & secondary)

Let $\mathcal{D}_b$ be the set of trading days that produced a valid IVRMSE for bucket $b$, and let $R_{b,d}$ denote that **daily IVRMSE** (for the chosen slice) on day $d$.

**Primary — Equal-day mean (each day counts once):**
$$
\overline{R}^{\text{equal}}_{b}
=\frac{1}{|\mathcal{D}_b|}\sum_{d\in\mathcal{D}_b} R_{b,d}.
$$

**Secondary — Pooled (contract-weighted) IVRMSE**  
(= pooling **all contracts** across the period in bucket $b$):
$$
\overline{R}^{\text{pooled}}_{b}
=\sqrt{\frac{\sum_{d\in\mathcal{D}_b}\sum_{i\in(b,d)}
\big(\Delta\sigma_{i}\big)^2}{\sum_{d\in\mathcal{D}_b} n_{b,d}}}\times 1000,
\qquad
\Delta\sigma_{i} =
\begin{cases}
\hat\sigma-\sigma^{\text{mkt}}_i,& \text{BS}\\[2pt]
\sigma^{\text{mdl}}_i-\sigma^{\text{mkt}}_i,& \text{JD/Heston}
\end{cases}
$$

**Equivalent “RMSE pooling” using only daily IVRMSE and counts:**
$$
\overline{R}^{\text{pooled}}_{b}
=\sqrt{\frac{\sum_{d\in\mathcal{D}_b} n_{b,d}\,\big(R_{b,d}/1000\big)^2}{\sum_{d\in\mathcal{D}_b} n_{b,d}}}\times 1000.
$$

**Units & notes.**  
- All IVRMSE values are reported $\times\,1000$ (as in the reference tables).  
- Days with **no valid bucket** are omitted from $\mathcal{D}_b$.  
- The **equal-day mean** is the recommended primary metric (prevents heavy-volume days from dominating); the **pooled** metric is a useful secondary check.


In [8]:
# ============================================================
# IVRMSE bookkeeping
# ============================================================

def _ivrmse_vs_constant_sigma(calls_bucket: pd.DataFrame, sig_hat: float) -> Dict[str, float]:
    """IVRMSE slices when model IV is constant σ̂ across strikes in bucket."""
    def rmse(sub: pd.DataFrame) -> float:
        if len(sub) == 0: return np.nan
        return float(np.sqrt(np.mean((sub["sigma_mkt_b76"].values - sig_hat)**2)))
    return {
        "whole": rmse(calls_bucket),
        "<1":    rmse(calls_bucket[calls_bucket["moneyness_F"] < 1.0]),
        ">1":    rmse(calls_bucket[calls_bucket["moneyness_F"] > 1.0]),
        ">1.03": rmse(calls_bucket[calls_bucket["moneyness_F"] > 1.03]),
    }

def _ivrmse_vs_model_prices(calls_bucket: pd.DataFrame, price_fn) -> Dict[str, float]:
    """
    Price with a model → invert to B76 IV → compare to market IVs.
    Keep only rows where inversion succeeds.
    """
    sig_model, keep = [], []
    for _, r0 in calls_bucket.iterrows():
        pm = price_fn(F=float(r0["F"]), K=float(r0["strike"]), tau=float(r0["tau"]), r=float(r0["r"]))
        s  = b76_iv_from_price(pm, float(r0["F"]), float(r0["strike"]), float(r0["tau"]), float(r0["r"]))
        if s is not None:
            sig_model.append(s); keep.append(True)
        else:
            keep.append(False)
    if not any(keep):
        return {"whole": np.nan, "<1": np.nan, ">1": np.nan, ">1.03": np.nan}
    cb = calls_bucket.loc[np.array(keep, dtype=bool)].copy()
    cb["sigma_model_b76"] = np.array(sig_model, dtype=float)

    def rmse(sub: pd.DataFrame) -> float:
        if len(sub) == 0: return np.nan
        return float(np.sqrt(np.mean((sub["sigma_model_b76"].values - sub["sigma_mkt_b76"].values)**2)))
    return {
        "whole": rmse(cb),
        "<1":    rmse(cb[cb["moneyness_F"] < 1.0]),
        ">1":    rmse(cb[cb["moneyness_F"] > 1.0]),
        ">1.03": rmse(cb[cb["moneyness_F"] > 1.03]),
    }

def _pooled_rmse_x1000(block: pd.DataFrame, iv_col: str, weight_col: str = "N") -> float:
    """
    Contract-weighted pooled RMSE across rows in `block`, ignoring NaNs.
    Returns NaN only if no valid (value, weight) pairs remain after masking
    or if total weight is zero.
    """
    import numpy as np
    x = block[iv_col].to_numpy(dtype=float)           # IVRMSE ×1000 (may contain NaN)
    w = block[weight_col].to_numpy(dtype=float)       # weights (contract counts)
    m = np.isfinite(x) & np.isfinite(w) & (w > 0)     # keep only valid pairs
    if not m.any():
        return np.nan
    x_raw = x[m] / 1000.0                             # back to raw RMSE
    w_use = w[m]
    return float(np.sqrt((w_use * (x_raw ** 2)).sum() / w_use.sum()) * 1000.0)

## 10. Period summarizer (daily loop, saving)

For each **trading day** in the selected period:

1. **Prepare calls & buckets**  
   - Clean quotes, form call–put pairs, infer $(F,DF,r)$ per expiry via parity.  
   - Compute market Black–76 IVs for calls.  
   - Map each contract to the nearest **maturity bucket** (e.g., 14d / 28d / 56d).

2. **Calibrate per bucket (price SSE)**  
   - **BS/Black–76:** fit a single $\hat\sigma$.  
   - **Merton JD:** fit $(\sigma,\lambda,\mu_J,\delta_J)$.  
   - **Heston SV:** fit $(\kappa,\theta,\sigma_v,\rho,v_0)$ using CF + Simpson integrals.

3. **Evaluate IVRMSE slices**  
   - **BS:** compare $\hat\sigma$ directly to market IVs.  
   - **JD/Heston:** price each strike, **invert to Black–76 IV**, compare to market IVs.  
   - Report slices: **Whole**, **$m<1$**, **$m>1$**, **$m>1.03$** (with $m=K/F$).

4. **Daily compact log (Whole × model)**  
   - After all buckets that day, print one line with the **pooled (contract-weighted) Whole IVRMSE** per model and total counts.

---

### Saved outputs (if `out_dir` is provided)

- **`daily_ivrmse.csv`** — rows per *(day, bucket)* with `N` and IVRMSE slices for each model.  
- **`equal_day_mean.csv`** — **primary** metric: equal-day mean IVRMSE per bucket.  
- **`pooled.csv`** — **secondary** metric: pooled (contract-weighted) IVRMSE per bucket.  
- **`daily_log.txt`** — one-line **pooled Whole** summary per day (quick progress/diagnostics).

The publication table is built from `equal_day_mean.csv` (primary) and cross-checked with `pooled.csv` (secondary).

In [9]:
# ============================================================
# Main entry: summarize symbol over a period with saving
# ============================================================

def summarize_symbol_period_ivrmse(
    df_all: pd.DataFrame,
    symbol: str = "SPY",
    start_date: str = "2025-04-01",
    end_date:   str = "2025-06-30",
    buckets: List[int] = [14, 28, 56],
    min_parity_pairs: int = 4,
    tau_floor_days: int = 3,
    run_bs: bool = True,
    run_jd: bool = True,
    run_heston: bool = True,
    show_progress: bool = True,
    print_daily: bool = True,
    out_dir: Optional[str] = None,
) -> Dict[str, pd.DataFrame]:
    """
    ONE summary for a symbol over a period (per bucket):
      • Primary  = equal-day mean IVRMSE (each day counts once)
      • Secondary= pooled/N-weighted IVRMSE (every contract counts)
    Saves: daily table, equal_day_mean, pooled, and a daily log (one line/day).
    """
    # Output folder & run config
    outp = None
    log_fp = None
    if out_dir is not None:
        outp = Path(out_dir)
        outp.mkdir(parents=True, exist_ok=True)
        log_fp = (outp / "daily_log.txt").open("w", encoding="utf-8")
        (outp / "run_config.txt").write_text(
            f"symbol={symbol}\\nperiod={start_date}..{end_date}\\n"
            f"buckets={buckets}\\nmin_parity_pairs={min_parity_pairs}\\n"
            f"tau_floor_days={tau_floor_days}\\nrun_bs={run_bs} run_jd={run_jd} run_heston={run_heston}\\n",
            encoding="utf-8"
        )

    # Filter to symbol & period
    df = df_all.copy()
    sym_col = "act_symbol" if "act_symbol" in df.columns else "symbol"
    if sym_col not in df.columns:
        if log_fp: log_fp.close()
        raise ValueError("Expected 'act_symbol' or 'symbol' in DataFrame.")
    df["date"] = pd.to_datetime(df["date"]).dt.normalize()
    mask = (df[sym_col] == symbol) & (df["date"] >= pd.Timestamp(start_date)) & (df["date"] <= pd.Timestamp(end_date))
    df = df.loc[mask].copy()
    if df.empty:
        if log_fp: log_fp.close()
        raise ValueError(f"No rows for {symbol} in [{start_date}, {end_date}].")

    # Bucket mapper using requested centers
    def assign_bucket_centers(tau_years: float) -> str:
        days = tau_years * 365.0
        i = int(np.argmin([abs(days - c) for c in buckets]))
        return f"{buckets[i]}d"

    # Iterate by day with progress
    days = sorted(df["date"].unique())
    iterator = days
    if show_progress:
        try:
            from tqdm.auto import tqdm
            iterator = tqdm(days, desc=f"{symbol} {pd.Timestamp(start_date).date()}→{pd.Timestamp(end_date).date()}")
        except Exception:
            pass

    daily_rows = []

    for day in iterator:
        df_day = df[df["date"] == day]
        calls, _ = prepare_calls_one_day_symbol(df_day, min_parity_pairs=min_parity_pairs, tau_floor_days=tau_floor_days)
        if calls.empty:
            msg = f"[{pd.Timestamp(day).date()}] no valid pairs/contracts after filters"
            if print_daily: print(msg)
            if log_fp: print(msg, file=log_fp)
            continue

        calls["bucket"] = calls["tau"].apply(assign_bucket_centers)

        day_rows = []
        for bucket, g in calls.groupby("bucket"):
            row = {"date": pd.Timestamp(day), "bucket": bucket, "N": int(len(g))}

            # BS/B76 baseline (1-σ)
            if run_bs:
                sig_hat = fit_sigma_bucket(g)
                ivs = _ivrmse_vs_constant_sigma(g, sig_hat)
                row.update({
                    "BS_IVRMSE_x1000_Whole":  ivs["whole"] * 1000 if np.isfinite(ivs["whole"]) else np.nan,
                    "BS_IVRMSE_x1000_<1":     ivs["<1"]    * 1000 if np.isfinite(ivs["<1"])    else np.nan,
                    "BS_IVRMSE_x1000_>1":     ivs[">1"]    * 1000 if np.isfinite(ivs[">1"])    else np.nan,
                    "BS_IVRMSE_x1000_>1.03":  ivs[">1.03"] * 1000 if np.isfinite(ivs[">1.03"]) else np.nan,
                })

            # JD baseline
            if run_jd:
                jd_params, _ = calibrate_jd_bucket(g)
                def jd_price(F,K,tau,r):
                    return merton_price_call_b76(F,K,tau,r, jd_params["sigma"], jd_params["lam"], jd_params["muJ"], jd_params["deltaJ"])
                ivs = _ivrmse_vs_model_prices(g, jd_price)
                row.update({
                    "JD_IVRMSE_x1000_Whole":  ivs["whole"] * 1000 if np.isfinite(ivs["whole"]) else np.nan,
                    "JD_IVRMSE_x1000_<1":     ivs["<1"]    * 1000 if np.isfinite(ivs["<1"])    else np.nan,
                    "JD_IVRMSE_x1000_>1":     ivs[">1"]    * 1000 if np.isfinite(ivs[">1"])    else np.nan,
                    "JD_IVRMSE_x1000_>1.03":  ivs[">1.03"] * 1000 if np.isfinite(ivs[">1.03"]) else np.nan,
                })

            # Heston baseline
            if run_heston:
                h_params, _ = calibrate_heston_bucket(g, u_max=100.0, n_points=501)  # stable Simpson settings
                def h_price(F,K,tau,r):
                    return heston_price_call(F,K,tau,r, h_params, u_max=100.0, n_points=501)
                ivs = _ivrmse_vs_model_prices(g, h_price)
                row.update({
                    "Heston_IVRMSE_x1000_Whole":  ivs["whole"] * 1000 if np.isfinite(ivs["whole"]) else np.nan,
                    "Heston_IVRMSE_x1000_<1":     ivs["<1"]    * 1000 if np.isfinite(ivs["<1"])    else np.nan,
                    "Heston_IVRMSE_x1000_>1":     ivs[">1"]    * 1000 if np.isfinite(ivs[">1"])    else np.nan,
                    "Heston_IVRMSE_x1000_>1.03":  ivs[">1.03"] * 1000 if np.isfinite(ivs[">1.03"]) else np.nan,
                })

            day_rows.append(row)

        if not day_rows:
            msg = f"[{pd.Timestamp(day).date()}] no valid buckets"
            if print_daily: print(msg)
            if log_fp: print(msg, file=log_fp)
            continue

        daily_rows.extend(day_rows)

        # One compact line per day — pooled across buckets (Whole slice only)
        msg_parts = []
        day_df = pd.DataFrame(day_rows)
        for model, col in [("BS", "BS_IVRMSE_x1000_Whole"),
                           ("JD", "JD_IVRMSE_x1000_Whole"),
                           ("Heston", "Heston_IVRMSE_x1000_Whole")]:
            if col in day_df.columns:
                pooled = _pooled_rmse_x1000(day_df[["N", col]].rename(columns={col: "val"}), "val")
                msg_parts.append(f"{model}={round(float(pooled),1) if pd.notna(pooled) else 'NA'}")
        msg = f"[{pd.Timestamp(day).date()}] pooled Whole x1000: " + ", ".join(msg_parts)
        if print_daily: print(msg)
        if log_fp: print(msg, file=log_fp)

    if log_fp: log_fp.close()

    if not daily_rows:
        raise RuntimeError("No valid (day,bucket) rows — check filters or min_parity_pairs/tau_floor_days.")

    daily = pd.DataFrame(daily_rows)

    # ---- Primary: equal-day mean per bucket (each day counts once) ----
    iv_cols = [c for c in daily.columns if c.endswith("_IVRMSE_x1000_Whole")
                                     or c.endswith("_IVRMSE_x1000_<1")
                                     or c.endswith("_IVRMSE_x1000_>1")
                                     or c.endswith("_IVRMSE_x1000_>1.03")]
    equal_day_mean = (daily.groupby("bucket", as_index=False)[iv_cols].mean())
    # Coverage diagnostics
    days_used = daily.groupby("bucket")["date"].nunique().rename("days_used").reset_index()
    N_total   = daily.groupby("bucket")["N"].sum().rename("N_total").reset_index()
    equal_day_mean = (equal_day_mean.merge(days_used, on="bucket")
                                   .merge(N_total, on="bucket")
                                   .sort_values("bucket"))

    # ---- Secondary: pooled/N-weighted per bucket ----
    pooled_rows = []
    for b, g in daily.groupby("bucket"):
        row = {"bucket": b, "days_used": int(g["date"].nunique()), "N_total": int(g["N"].sum())}
        for col in iv_cols:
            row[col] = _pooled_rmse_x1000(g[["N", col]].rename(columns={col: "val"}), "val")
        pooled_rows.append(row)
    pooled = pd.DataFrame(pooled_rows).sort_values("bucket")

    # Save outputs
    if outp is not None:
        daily.to_csv(outp / "daily_ivrmse.csv", index=False)
        equal_day_mean.to_csv(outp / "equal_day_mean.csv", index=False)
        pooled.to_csv(outp / "pooled.csv", index=False)

    return {"daily": daily, "equal_day_mean": equal_day_mean, "pooled": pooled}

## 11. Publication-style table

**Goal.** Present model accuracy (IVRMSE ×1000) in a compact, reader-friendly table that mirrors common practice in the literature.

**Layout.**
- **Rows:** by **moneyness slice** (Whole, $m<1$, $m>1$, $m>1.03$) and **maturity bucket** (e.g., 14d / 28d / 56d).  
- **Columns:** **BS**, **JD**, **SV (Heston)** — each entry is IVRMSE ×1000.

**Source of values.**
- Use the **primary** metric (the **equal-day mean** per bucket) for the table.  
- Optionally report the **secondary** metric (contract-weighted **pooled** IVRMSE) in an appendix or robustness table.

**Rendering & export.**
- If `out_dir` is provided, save both:
  - **CSV**: machine-readable table.
  - **Markdown**: human-readable version with formatting.
- In the Markdown table, **bold the minimum in each row** (best model for that slice/bucket) to improve readability.

**Notes.**
- All numbers are **scaled by 1000** (e.g., 7.5 → IVRMSE of 0.0075).  
- Days/buckets with no valid quotes or failed inversions are omitted from averaging.  
- The table reflects the **same data filters** used in the baselines (min paired strikes, $\tau$ floor, etc.).

In [10]:
# ============================================================
# Publication-style table (CSV + Markdown with bold minima)
# ============================================================

def make_publication_table(res: Dict[str, pd.DataFrame],
                           symbol: str = "SPY",
                           measure: str = "equal",          # "equal" (primary) or "pooled" (secondary)
                           buckets: List[int] = (14, 28, 56),
                           decimals: int = 2,
                           out_dir: Optional[str] = None,
                           basename: str = "table_ivrmse") -> pd.DataFrame:
    """
    Build a paper-style table matching the reference format:
      Sections: Whole sample / Moneyness <1 / >1 / >1.03
      Rows: one per bucket like 'SPY (τ=14d)'
      Columns: BS, JD, SV (Heston)
    Saves CSV + Markdown (bolds the row minimum) if out_dir is set.
    """
    if measure not in ("equal", "pooled"):
        raise ValueError("measure must be 'equal' or 'pooled'.")

    df_src = res["equal_day_mean"] if measure == "equal" else res["pooled"]
    if df_src is None or df_src.empty:
        raise ValueError("Source summary is empty.")

    present = set(df_src.columns)
    models = [m for m, prefix in [("BS","BS"), ("JD","JD"), ("SV","Heston")]
              if any(col.startswith(f"{prefix}_IVRMSE_x1000") for col in present)]
    model_to_prefix = {"BS":"BS", "JD":"JD", "SV":"Heston"}

    sections = [("Whole sample", "Whole"),
                ("Moneyness <1", "<1"),
                ("Moneyness >1", ">1"),
                ("Moneyness >1.03", ">1.03")]
    bucket_labels = [f"{d}d" for d in buckets]

    rows = []
    for section_name, suffix in sections:
        for d in bucket_labels:
            row = {"Moneyness": section_name, "Asset": f"{symbol} (τ={d})"}
            sub = df_src[df_src["bucket"] == d]
            if sub.empty:
                for m in models: row[m] = np.nan
            else:
                for m in models:
                    prefix = model_to_prefix[m]
                    col = f"{prefix}_IVRMSE_x1000_{suffix}"
                    row[m] = float(sub[col].iloc[0]) if col in sub.columns and pd.notna(sub[col].iloc[0]) else np.nan
            rows.append(row)

    table = pd.DataFrame(rows)
    for m in models:
        table[m] = table[m].round(decimals)

    if out_dir:
        outp = Path(out_dir); outp.mkdir(parents=True, exist_ok=True)
        csv_path = outp / f"{basename}_{measure}.csv"
        table.to_csv(csv_path, index=False)

        # Markdown with bold minimum per row
        md_rows = []
        header = ["Moneyness", "Asset"] + models
        md_rows.append("| " + " | ".join(header) + " |")
        md_rows.append("| " + " | ".join(["---"]*len(header)) + " |")
        for _, r in table.iterrows():
            vals = [r[m] for m in models]
            not_nan = [i for i,v in enumerate(vals) if pd.notna(v)]
            best_idx = None
            if not_nan:
                best_idx = min(not_nan, key=lambda i: vals[i])
            cells = [str(r["Moneyness"]), str(r["Asset"])]
            for i, m in enumerate(models):
                v = r[m]
                if pd.isna(v):
                    cells.append("")
                else:
                    s = f"{v:.{decimals}f}"
                    cells.append(f"**{s}**" if i == best_idx else s)
            md_rows.append("| " + " | ".join(cells) + " |")
        md_text = "\\n".join(md_rows)
        md_path = outp / f"{basename}_{measure}.md"
        md_path.write_text(md_text, encoding="utf-8")
        print(f"Saved: {csv_path}")
        print(f"Saved: {md_path}")

    return table

### SPY (Benchmark) — Q2 2025 IVRMSE
 
SPY is the SPDR S&P 500 ETF Trust, a widely traded ETF that tracks the S&P 500 index (U.S. large-cap equities). Its options are among the most liquid in the market, making SPY a standard benchmark for model comparisons.
 
Here we report **IVRMSE ×1000** for SPY options over **Q2 2025** (April–June), summarized by maturity buckets and moneyness.

In [11]:
df = pd.read_csv("./SPY Options 2025.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23130 entries, 0 to 23129
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        23130 non-null  object 
 1   act_symbol  23130 non-null  object 
 2   expiration  23130 non-null  object 
 3   strike      23130 non-null  float64
 4   call_put    23130 non-null  object 
 5   bid         23130 non-null  float64
 6   ask         23130 non-null  float64
 7   vol         23130 non-null  float64
 8   delta       23130 non-null  float64
 9   gamma       23130 non-null  float64
 10  theta       23130 non-null  float64
 11  vega        23130 non-null  float64
 12  rho         23130 non-null  float64
dtypes: float64(9), object(4)
memory usage: 2.3+ MB


In [12]:
res = summarize_symbol_period_ivrmse(
    df_all=df,
    symbol="SPY",
    start_date="2025-04-01",
    end_date="2025-06-30",
    buckets=[14,28,56],
    min_parity_pairs=4,
    tau_floor_days=3,
    run_bs=True, run_jd=True, run_heston=True,
    show_progress=True,
    print_daily=True,
    out_dir="SPY_25Q2_baseline"      # outputs saved here
)

# PRIMARY (paper): equal-day mean table
tbl_equal = make_publication_table(res, symbol="SPY",
                                   measure="equal",
                                   buckets=[14,28,56],
                                   decimals=2,
                                   out_dir="SPY_25Q2_baseline",
                                   basename="table_ivrmse")

# SECONDARY (robustness): pooled table
tbl_pooled = make_publication_table(res, symbol="SPY",
                                    measure="pooled",
                                    buckets=[14,28,56],
                                    decimals=2,
                                    out_dir="SPY_25Q2_baseline",
                                    basename="table_ivrmse")

print(tbl_equal)
print(tbl_pooled)

SPY 2025-04-01→2025-06-30:   0%|          | 0/65 [00:00<?, ?it/s]

[2025-04-01] pooled Whole x1000: BS=58.8, JD=36.6, Heston=68.3
[2025-04-02] pooled Whole x1000: BS=64.0, JD=48.5, Heston=72.4
[2025-04-03] pooled Whole x1000: BS=81.8, JD=25.1, Heston=35.4
[2025-04-04] pooled Whole x1000: BS=112.1, JD=11.7, Heston=21.0
[2025-04-07] pooled Whole x1000: BS=116.5, JD=11.4, Heston=9.1
[2025-04-08] pooled Whole x1000: BS=121.1, JD=11.6, Heston=18.6
[2025-04-09] pooled Whole x1000: BS=79.4, JD=16.4, Heston=27.5
[2025-04-10] pooled Whole x1000: BS=112.9, JD=16.4, Heston=34.7
[2025-04-11] pooled Whole x1000: BS=110.5, JD=21.3, Heston=28.1
[2025-04-14] pooled Whole x1000: BS=87.9, JD=32.8, Heston=56.7
[2025-04-15] pooled Whole x1000: BS=90.1, JD=34.2, Heston=37.7
[2025-04-16] pooled Whole x1000: BS=87.2, JD=28.9, Heston=45.2
[2025-04-17] pooled Whole x1000: BS=81.0, JD=32.1, Heston=40.9
[2025-04-18] pooled Whole x1000: BS=82.2, JD=29.1, Heston=46.8
[2025-04-21] pooled Whole x1000: BS=78.2, JD=21.8, Heston=38.2
[2025-04-22] pooled Whole x1000: BS=76.9, JD=25.0, 

### XOP — Q2 2025 IVRMSE

**XOP** is the SPDR S&P Oil & Gas Exploration & Production ETF, tracking an equal-weighted index of U.S. E&P companies in the energy sector. 
 
Here we report **IVRMSE ×1000** for XOP options over **Q2 2025** (April–June), summarized by maturity buckets and moneyness.

In [13]:
df = pd.read_csv("./XOP Options 2025.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23106 entries, 0 to 23105
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        23106 non-null  object 
 1   act_symbol  23106 non-null  object 
 2   expiration  23106 non-null  object 
 3   strike      23106 non-null  float64
 4   call_put    23106 non-null  object 
 5   bid         23106 non-null  float64
 6   ask         23106 non-null  float64
 7   vol         23106 non-null  float64
 8   delta       23106 non-null  float64
 9   gamma       23106 non-null  float64
 10  theta       23106 non-null  float64
 11  vega        23106 non-null  float64
 12  rho         23106 non-null  float64
dtypes: float64(9), object(4)
memory usage: 2.3+ MB


In [14]:
res = summarize_symbol_period_ivrmse(
    df_all=df,
    symbol="XOP",
    start_date="2025-04-01",
    end_date="2025-06-30",
    buckets=[14,28,56],
    min_parity_pairs=4,
    tau_floor_days=3,
    run_bs=True, run_jd=True, run_heston=True,
    show_progress=True,
    print_daily=True,
    out_dir="XOP_25Q2_baseline"      # outputs saved here
)

# PRIMARY (paper): equal-day mean table
tbl_equal = make_publication_table(res, symbol="XOP",
                                   measure="equal",
                                   buckets=[14,28,56],
                                   decimals=2,
                                   out_dir="XOP_25Q2_baseline",
                                   basename="table_ivrmse")

# SECONDARY (robustness): pooled table
tbl_pooled = make_publication_table(res, symbol="XOP",
                                    measure="pooled",
                                    buckets=[14,28,56],
                                    decimals=2,
                                    out_dir="XOP_25Q2_baseline",
                                    basename="table_ivrmse")

print(tbl_equal)
print(tbl_pooled)

XOP 2025-04-01→2025-06-30:   0%|          | 0/64 [00:00<?, ?it/s]

[2025-04-01] pooled Whole x1000: BS=71.8, JD=91.6, Heston=95.0
[2025-04-02] pooled Whole x1000: BS=52.6, JD=43.3, Heston=67.4
[2025-04-03] pooled Whole x1000: BS=72.0, JD=65.1, Heston=66.8
[2025-04-04] pooled Whole x1000: BS=51.5, JD=26.0, Heston=26.0
[2025-04-07] pooled Whole x1000: BS=63.9, JD=22.4, Heston=28.4
[2025-04-08] pooled Whole x1000: BS=63.1, JD=13.4, Heston=13.4
[2025-04-09] pooled Whole x1000: BS=90.6, JD=43.7, Heston=50.8
[2025-04-10] pooled Whole x1000: BS=77.5, JD=18.7, Heston=26.5
[2025-04-11] pooled Whole x1000: BS=93.2, JD=28.6, Heston=54.5
[2025-04-14] pooled Whole x1000: BS=104.5, JD=26.4, Heston=69.3
[2025-04-15] pooled Whole x1000: BS=70.8, JD=29.6, Heston=32.4
[2025-04-16] pooled Whole x1000: BS=87.5, JD=35.7, Heston=48.0
[2025-04-17] pooled Whole x1000: BS=51.3, JD=31.8, Heston=27.3
[2025-04-18] pooled Whole x1000: BS=52.5, JD=32.8, Heston=28.1
[2025-04-21] pooled Whole x1000: BS=104.8, JD=30.5, Heston=71.1
[2025-04-22] pooled Whole x1000: BS=74.3, JD=48.1, He